In [3]:
import pandas as pd
import numpy as np 
eps = np.finfo(float).eps
from numpy import log2 as log

def find_ent(df) :
    Class = df.keys()[-1]
    values = df[Class].unique()
    ent = 0
    for val in values :
        fraction = df[Class].value_counts()[val]/len(df)
        ent+=-fraction*log(fraction)
        
    return ent

def find_ent_att(df,att) :
    Class = df.keys()[-1]
    variables = df[att].unique()
    tar_vars = df[Class].unique()
    ent2 =0
    for var in variables :
        ent =0
        for tar_var in tar_vars :
            num = len(df[att][df[att] == var][df[Class] == tar_var])
            den = len(df[att][df[att] == var])
            fraction1 = num/(den+eps)
            ent += -fraction1*log(fraction1+eps)
        fraction2 = den/len(df)
        ent2 +=-fraction2*ent
    return abs(ent2)

def getSuitable(df,node,val):
    return df[df[node] == val].reset_index(drop = True)

def find_winner(df) :
    ig = []
    for key in df.keys()[:-1] :
        ig.append(find_ent(df)-find_ent_att(df,key))
    
    return df.keys()[:-1][np.argmax(ig)]

def build_tree(df,tree=None):
    node = find_winner(df)
    values = df[node].unique()
    if tree ==  None :
        tree = {}
        tree[node] = {}
        
    for val in values :
        suitable = getSuitable(df,node,val)
        cValue,cnt = np.unique(suitable['play'],return_counts = True)
        if len(cnt) == 1 :
            tree[node][val] = cValue[0]
        else :
            tree[node][val] = build_tree(suitable)
            
    return tree

dataset = pd.read_csv('tennis.csv')
tree = build_tree(dataset)
import pprint
pprint.pprint(tree)

{'outlook': {'overcast': 'yes',
             'rain': {'wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'hum': {'high': 'no', 'normal': 'yes'}}}}
